In [ ]:
%pip install -q dagshub 'mlflow>=2,<3'

In [ ]:
import dagshub
dagshub.init(repo_owner='AhmedNasir7', repo_name='mlflow_assignment_flask', mlflow=True)

In [ ]:
!pip install kagglehub


In [ ]:
import kagglehub

path = kagglehub.dataset_download("tanlikesmath/the-oxfordiiit-pet-dataset")
print("Path to dataset files:", path)

In [ ]:
import pathlib

data_dir = pathlib.Path(path)
print("Data directory exists?", data_dir.exists())
print("Contents:", list(data_dir.iterdir())[:10])


In [ ]:
import tensorflow as tf

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir / "images",
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir / "images",
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(224, 224),
    batch_size=32
)


In [45]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt


In [46]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(224, 224, 3)),
    layers.Conv2D(16, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [ ]:
import mlflow
import mlflow.tensorflow

with mlflow.start_run():
    model.fit(train_ds, validation_data=val_ds, epochs=5)

    mlflow.log_param("epochs", 5)
    mlflow.log_param("optimizer", "adam")

    loss, acc = model.evaluate(val_ds)
    mlflow.log_metric("val_loss", loss)
    mlflow.log_metric("val_accuracy", acc)

    mlflow.tensorflow.log_model(model, artifact_path="pet_model")
